In [4]:
import torch
# from datasets import load_dataset
import pandas as pd
import torch.utils.data as Data

# 定义数据集


class Dataset(torch.utils.data.Dataset):
    # 继承torch的Dataset类，并且需要重写它的这三个方法

    # 作用与所有构造函数都一样，初始化一个类的实例。定义类的实际属性
    def __init__(self):
        #     self.dataset = load_dataset(path='csv', data_files='train.csv', split=split)
        self.dataset = pd.read_csv('train.csv')


    # 相当于返回整体数据data的shape[0]，即给item的递增指定一个范围。
    def __len__(self):
        return self.dataset.shape[0]

    # 根据item的值取出数据。item实际上就是索引值，会由Dataloader自动从0一直递增到__len__中取出的值
    def __getitem__(self, index):
        text = self.dataset.loc[index, :]['text']
        label = self.dataset.loc[index, :]['label']

        return text, label


datasets = Dataset()
dataloader = Data.DataLoader(datasets, batch_size = 100)
for  (inputdata, target) in dataloader:
    print(inputdata)



('歌词是来不及，但是每一个字都在说着来得及。', '原版像在下沉，玲花一开口，感觉从海底浮上来了', '这是两位佛，在渡众生啊，这样温柔却这样有力量。', '凤凰传奇 永远的神', '有种草原呼麦的感觉', '歌词真好', '来得及', '上岸！', '希望成都49中的那个男生可以听到这首歌', '快去关注凤凰传奇！！！', '凤凰传奇是一种温暖的力量', '花朝我放电了', '我会上岸的！', '非常棒，1201.4 W打卡', '每日一刷', '曾毅这两年发现钱不好赚了', '真的好棒， 点赞', '这才叫真翻唱，对原唱的另一种解读和升华！！！', '明天是个好天气！！！', '看完一遍，又多了两千播放量', '舒服了今天', '春晚好吧', '初级必过！！', '上岸', '凤凰传奇yyds', '好听', '词曲好棒', '凤凰传奇yyds！', '698人正在看~~~', '大鱼！', '曾毅太帅了', '每天一遍！', '来得及！东林等我', '上岸，加油', '来得及来得及 我要好好学习', '来得及来得及 我要好好学习！', '考研上岸', '上岸！', '明天是个好天气！！！', '会计上岸', '活着看四季', '来得及的一定来得及的', 'zl我永远爱你', '人间值得', '明天是个好天气！今天你已经很棒了！', '1200万播放？牛啊', '网易云我买了', '好听啊', '全体起立！', '来了', '等我娶了她我再回来看', '你永远可以相信凤凰传奇', '原唱:海底 凤凰翻唱:海底捞', '好听', '梵音救赎的感觉', '我要努力努力更努力', '大鱼bgmyyds', '佛光普照', '希望能上川警', '我想考上北京师范大学去见他！', '我知道活成自己想要的样子，不容易，但是尽力朝那个方向去努力吧', '我可以', '公务员面试上岸', '第一次来', '一遍真的够么', '太棒了', '单曲再循环', '第100遍', '“明天是个好天气”', 'yyds', '上岸', '助力1200w', '明天是个好天气', '每天亿遍', '专升本希望有好成绩', '单曲再循环', '我又来了', '永远滴神', '过六级', '起鸡皮疙瘩了', '来得及，专升本一定上岸', '蒙古海军', '1200打卡', '我看到佛光在他们

In [ ]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

In [ ]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=50,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels

In [4]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

#模型试算
out = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

out.last_hidden_state.shape

ModuleNotFoundError: No module named 'transformers'

In [7]:
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])

        out = out.softmax(dim=1)

        return out


model = Model()

model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids).shape

NameError: name 'input_ids' is not defined

In [ ]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for i, (input_ids, attention_mask, token_type_ids,
        labels) in enumerate(loader):
    out = model(input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i % 5 == 0:
        out = out.argmax(dim=1)
        accuracy = (out == labels).sum().item() / len(labels)

        print(i, loss.item(), accuracy)

    if i == 300:
        break

In [ ]:
#测试
dataset2 = Dataset('test')
def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset2,
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)

    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader_test):

        if i == 5:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    print(correct / total)


test()